In [1]:
import numpy as np
import tensorflow as tf
import time
import gym

In [2]:
env = gym.make("CartPole-v1")

d:\Documents\Projects\CartPole_NN_Policy\mylib11\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [3]:
SEED = 42
tf.random.set_seed(SEED)

In [4]:
layers = [tf.keras.layers.Dense(5, activation="relu"),
tf.keras.layers.Dense(1, activation="sigmoid")      #gives left probability neans 1 if 0 that means right
]

model = tf.keras.Sequential(layers)

In [5]:
eg= np.array([1,2,3])
eg

array([1, 2, 3])

In [6]:
eg[np.newaxis]

array([[1, 2, 3]])

exploration an option before leaving the current action.

In [7]:
def pg_policy(observation, model):
    left_prob = model.predict(observation[np.newaxis])      #gives probability value between 0 and 1
    action =  int(np.random.rand()>left_prob)               #value{0,1} gives exploration vs exploitation
    return action

## Policy Gradients
Optimize learnable parameters of policy by following the gradients towards higher reward (maximizing reward)

## steps
let the NN play the game multiple times and at every step just calculate the gradients (wrt reward) but dont apply it immidiately.
Once you have completed several episodes then compute the actions using discounted method.
result of previous step 2 can +ve or -ve

In [9]:
def play_one_step(env, observation, model, loss_fn):
    with tf.GradientTape() as tape:
        left_prabability = model(observation[np.newaxis])
        action = (tf.random.uniform([1,1]) > left_prabability) # True and False
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32) # 
        loss = tf.reduce_mean(loss_fn(y_target, left_prabability)) 

    grads = tape.gradient(loss, model.trainable_variables) # dc/dw
    new_observation, reward, done, info = env.step(int(action))
    return new_observation, reward, done, grads

In [10]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = list()
    all_grads = list()
    for episode in range(n_episodes):
        current_rewards = list()
        current_grads = list()
        observation = env.reset()
        for step in range(n_max_steps):
            observation, reward, done, grads = play_one_step(env, observation, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

In [11]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    N = len(rewards)
    for step in range(N - 2, -1, -1):
        # a_n + a_n+1*gamma
        discounted[step] = discounted[step] + discounted[step + 1] * discount_factor
    return discounted

In [12]:
arr = [10, 0, -50]
discount_rewards(arr, 0.8)

array([-22, -40, -50])

In [13]:
x = np.array([1,2])
np.concatenate([x,x])

array([1, 2, 1, 2])

In [14]:
def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = list()
    for reward in all_rewards:
        # discounted rewards
        drs = discount_rewards(reward, discount_factor)
        all_discounted_rewards.append(drs)

    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()

    normalize_rewards = list()
    for discounted_rewards in all_discounted_rewards:
        nrs = (discounted_rewards - reward_mean) / reward_std
        normalize_rewards.append(nrs)
    return normalize_rewards

In [15]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_factor = 0.95
learning_rate = 0.01

In [16]:
obs = env.reset(seed=SEED)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.binary_crossentropy

In [17]:
r1 = [1,2,3]
r2 = [-1,-2,3]
all_rewards_1 = [r1, r2]
list(map(sum, all_rewards_1))

[6, 0]

In [18]:
sum(map(sum, all_rewards_1))

6

In [19]:
arr = [[1,2,3], [3,4,5]]
tf.reduce_mean(arr, axis=0)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4])>

In [20]:
for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn
    )
    total_rewards = sum(map(sum, all_rewards))
    print(f"Iteration: {iteration + 1}/{n_iterations}",
    f"mean rewards: {total_rewards/n_episodes_per_update}"
    )
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_factor)

    all_mean_grads = list()
    # Weight of 5 hidden nodes, bias for 5 nodes, w for output node, bias for output node
    N = len(model.trainable_variables)
    for var_index in range(N):
        temp_reduce_mean = list()
        for episode_index, final_rewards in enumerate(all_final_rewards): # rewards for every episode
            for step, final_reward in enumerate(final_rewards): # several steps
                result = final_reward * all_grads[episode_index][step][var_index]
                temp_reduce_mean.append(result)
        mean_grads = tf.reduce_mean(temp_reduce_mean, axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

Iteration: 1/150 mean rewards: 21.6
Iteration: 2/150 mean rewards: 17.7
Iteration: 3/150 mean rewards: 33.3
Iteration: 4/150 mean rewards: 29.3
Iteration: 5/150 mean rewards: 32.5
Iteration: 6/150 mean rewards: 33.5
Iteration: 7/150 mean rewards: 32.2
Iteration: 8/150 mean rewards: 24.7
Iteration: 9/150 mean rewards: 25.6
Iteration: 10/150 mean rewards: 32.5
Iteration: 11/150 mean rewards: 36.1
Iteration: 12/150 mean rewards: 32.5
Iteration: 13/150 mean rewards: 30.3
Iteration: 14/150 mean rewards: 31.7
Iteration: 15/150 mean rewards: 45.9
Iteration: 16/150 mean rewards: 36.9
Iteration: 17/150 mean rewards: 43.2
Iteration: 18/150 mean rewards: 34.5
Iteration: 19/150 mean rewards: 38.1
Iteration: 20/150 mean rewards: 29.3
Iteration: 21/150 mean rewards: 45.1
Iteration: 22/150 mean rewards: 34.2
Iteration: 23/150 mean rewards: 29.6
Iteration: 24/150 mean rewards: 32.5
Iteration: 25/150 mean rewards: 45.3
Iteration: 26/150 mean rewards: 43.5
Iteration: 27/150 mean rewards: 47.7
Iteration:

In [21]:
import re
import time

unique_name = re.sub(r"[\s+:]", "_", time.asctime())
model_name = f"model_at_{unique_name}_.h5"
model.save(model_name)
print(f"model is saved as '{model_name}'")

model is saved as 'model_at_Tue_Jul_12_17_35_24_2022_.h5'


In [22]:
r1 = [1,2,3]
r2 = [-1,-2,3]
list(zip(r1, r2))

[(1, -1), (2, -2), (3, 3)]

In [23]:
def show_one_episode(policy, model, n_max_steps=500, seed=42):
    env = gym.make("CartPole-v1")
    obs = env.reset()
    for step in range(n_max_steps):
        env.render()
        action = policy(obs, model)
        obs, reward, done, info = env.step(action)
        if done:
            break
    env.close()
    return step, obs

show_one_episode(pg_policy, model)

d:\Documents\Projects\CartPole_NN_Policy\mylib11\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


1/1 [==============================] - 0s 13ms/step


(211, array([2.175206  , 0.7111561 , 0.21600634, 0.9366191 ], dtype=float32))

In [24]:
def basic_policy(obs, model):
    PoleAngle = obs[2] 
    if PoleAngle < 0: # falling left
        return 0 # Move left
    return 1 

In [25]:

show_one_episode(basic_policy, model)

(34,
 array([ 0.20117114,  0.54828346, -0.2290762 , -0.78777736], dtype=float32))

In [26]:
show_one_episode(pg_policy, model)

1/1 [==============================] - 0s 13ms/step


(149,
 array([-2.4206684 , -2.1997023 ,  0.01727703,  0.36243927], dtype=float32))